# Génération de texte

Les données viennent du site https://jeunesse.short-edition.com/categorie/6-8 contenant de multiples histoires pour les enfants de 6 à 8 ans.

L'objectif est de générer des mots afin de construire de courtes phrases à partir du début que l'on donne.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import tensorflow as tf

In [ ]:
filename = "/content/drive/MyDrive/Projet nlp histoire/histoire.txt"

In [ ]:
with open(filename) as f:
  lines = f.readlines()

In [ ]:
lines[:5]

["J'ai peur. Il fait froid. Je me serre contre la fourrure de Jacquot. C'est notre chien. Adélaïde m'a dit de ne pas le quitter lorsqu'elle est partie avec Louise. Elle m'a laissé dans le jardin derrière la maison. « Reste avec Jacquot. Je vais chercher les clefs. Je reviens très vite ». Adé c'est notre babysitter. Elle vient nous chercher tous les soirs à l'école, récupère Louise dans la cour de la maternelle et moi à la sortie du CP. Elle s'occupe de nous jusqu'à ce que Papa ou Maman rentre.\n",
 "On s'amuse bien avec Adé. On joue à aller faire les courses : on prend le bus jusqu'en ville. On ne paie pas. Je n'aime pas ça, c'est interdit. Adé se moque : « Quel trouillard ! ». Dans le magasin, Adé nous achète des bonbons et on l'attend pendant qu'elle essaie des robes, des jupes, des Tshirts. Ou du maquillage. Deux fois on a « sonné » en sortant du magasin. Un monsieur nous a couru après. Adé a dû montrer ce qu'il y avait dans son sac. Elle avait oublié de payer du rouge à lèvres. Ell

In [ ]:
new_list = list()
for phrase in lines:
  temp = phrase.split('. ')
  for ph in temp:
    new_list.append(ph)

In [ ]:
new_list[:5]

["J'ai peur",
 'Il fait froid',
 'Je me serre contre la fourrure de Jacquot',
 "C'est notre chien",
 "Adélaïde m'a dit de ne pas le quitter lorsqu'elle est partie avec Louise"]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(new_list)

In [ ]:
total_mot = len(tokenizer.word_index) + 1

Pour connaitre la longueur moyenne des phrases:

In [ ]:
seq = tokenizer.texts_to_sequences(new_list)

In [ ]:
liste = list()
for longg in seq:
  liste.append(len(longg))

In [ ]:
np.mean(liste)

9.569959488937364

In [ ]:
input_sequence = list()
for line in new_list:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequence.append(n_gram_sequence)

In [ ]:
padded = pad_sequences(input_sequence, maxlen = 10, padding = 'pre')

In [ ]:
input = padded[:,:-1]
target = padded[:,-1]

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((input, target)).shuffle(1000).batch(128, drop_remainder=True).prefetch(1)

In [ ]:
model = Sequential([
                    Embedding(total_mot, 300, input_length=9),
                    Bidirectional(LSTM(10, return_sequences=True)),
                    LSTM(100),
                    Dense(total_mot/4, activation='relu'),
                    Dense(total_mot, activation='softmax')
])
model.compile(optimizer=Adam(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history =  model.fit(train_data, epochs=200)

Epoch 1/200
219/219 [==============================] - 53s 241ms/step - loss: 7.0224 - accuracy: 0.0380
Epoch 2/200
219/219 [==============================] - 41s 189ms/step - loss: 6.0775 - accuracy: 0.0605
Epoch 3/200
219/219 [==============================] - 43s 198ms/step - loss: 5.6754 - accuracy: 0.0826
Epoch 4/200
219/219 [==============================] - 43s 198ms/step - loss: 5.3867 - accuracy: 0.0954
Epoch 5/200
219/219 [==============================] - 46s 210ms/step - loss: 5.1677 - accuracy: 0.1047
Epoch 6/200
219/219 [==============================] - 45s 207ms/step - loss: 4.9878 - accuracy: 0.1128
Epoch 7/200
219/219 [==============================] - 46s 211ms/step - loss: 4.8405 - accuracy: 0.1244
Epoch 8/200
219/219 [==============================] - 48s 220ms/step - loss: 4.6981 - accuracy: 0.1318
Epoch 9/200
219/219 [==============================] - 47s 216ms/step - loss: 4.5685 - accuracy: 0.1368
Epoch 10/200
219/219 [==============================] - 47s 213m

In [ ]:
model.save('model_nlp.h5')

In [ ]:
def pred(seed_text,next_mot):
  for _ in range(next_mot):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=9, padding='pre')
    predicted = np.argmax(model(token_list))
    output_w = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_w = word
        break
    seed_text += " " + output_w
  return seed_text

In [ ]:
pred('je vais',3)

'je vais chercher les copains'

In [ ]:
pred('Il cherche', 3)

'Il cherche zéon à table'

In [ ]:
pred("C'était",3)

"C'était le père noël"

In [ ]:
pred("Il faut",5)

'Il faut bizarre de penser avec interdiction'

In [ ]:
pred("Il ne faut pas",11)

'Il ne faut pas crier non plus ça risque de lui faire de son aile'

In [ ]:
pred("Elle",4)

'Elle a peur du noir'

In [ ]:
pred("Je ne",5)

"Je ne suis pas d'accord elle marche"